In [4]:
""" Zonal statistics for basin area. Export in table format.
-------------------------------------------------------------------------------
Calculate the total area per basin in m2 for 5min and 30s resolution and for
level 00 and level 06 hydrobasins. Table is stored as pickle on S3 and postGIS.


Author: Rutger Hofste
Date: 20180420
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name


Returns:


"""

# Input Parameters
TESTING = 1
SCRIPT_NAME = "Y2018M04D20_RH_Zonal_Stats_Area_EE_V01"
EE_INPUT_ZONES_PATH = "projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V01"
EE_INPUT_VALUES_PATH = "projects/WRI-Aquaduct/PCRGlobWB20_Aux_V02"
INPUT_VERSION_ZONES = 4
INPUT_VERSION_VALUES = 5
OUTPUT_VERSION = 1

gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input ee zones: " +  EE_INPUT_ZONES_PATH +
      "\nInput ee values: " + EE_INPUT_VALUES_PATH +
      "\nOutput gcs: " + gcs_output_path)




Input ee zones: projects/WRI-Aquaduct/Y2018M04D20_RH_Ingest_HydroBasins_GCS_EE_V01/output_V01
Input ee values: projects/WRI-Aquaduct/PCRGlobWB20_Aux_V02
Output gcs: gs://aqueduct30_v01/Y2018M04D20_RH_Zonal_Stats_Area_EE_V01/output_V01/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M04D20 UTC 15:23


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import ee
import aqueduct3
ee.Initialize()





In [5]:
geometry = aqueduct3.earthengine.get_global_geometry(TESTING)

In [7]:
spatial_resolutions = ["5min","30s"]
pfaf_levels = ["06","00"]

for spatial_resolution in spatial_resolutions:
    for pfaf_level in pfaf_levels:
        print(spatial_resolution,pfaf_level)
        i_zones_asset_id = "{}/hybas_lev{}_v1c_merged_fiona_{}_V{:02.0f}".format(EE_INPUT_ZONES_PATH,pfaf_level,spatial_resolution,INPUT_VERSION_ZONES)
        i_values_asset_id = "{}/global_area_m2_{}_V{:02.0f}".format(EE_INPUT_VALUES_PATH,spatial_resolution,INPUT_VERSION_VALUES)
        total_image = ee.Image(i_values_asset_id).addBands(ee.Image(i_zones_asset_id))
        
        

5min 06
5min 00
30s 06
30s 00


In [13]:
reducer = ee.Reducer.mean().combine(reducer2= ee.Reducer.count(), sharedInputs= True).group(groupField=1, groupName= "zones")

In [14]:
test = total_image.reduceRegion(
        geometry = geometry,
        reducer= reducer,
        scale= 1000,
        maxPixels=1e10
      ).get("groups")

In [15]:
test.getInfo()

[{'count': 3, 'mean': 853825.719512195, 'zones': 133303000100.0},
 {'count': 164, 'mean': 853898.5193630112, 'zones': 133303000200.0},
 {'count': 90, 'mean': 854005.2395833334, 'zones': 133304010100.0},
 {'count': 188, 'mean': 854052.7895611703, 'zones': 133304010200.0},
 {'count': 74, 'mean': 853975.4763513514, 'zones': 133304010300.0},
 {'count': 156, 'mean': 854018.9657674325, 'zones': 133304020000.0},
 {'count': 145, 'mean': 853933.8498971193, 'zones': 133304030100.0},
 {'count': 10, 'mean': 853881.0845, 'zones': 133304030200.0},
 {'count': 1, 'mean': 853926.4375, 'zones': 133304030300.0},
 {'count': 17, 'mean': 854081.194143701, 'zones': 133304052000.0},
 {'count': 12, 'mean': 854142.853550296, 'zones': 133304054000.0},
 {'count': 18, 'mean': 854003.9618055555, 'zones': 133305000100.0},
 {'count': 121, 'mean': 854047.9070247934, 'zones': 133305000200.0},
 {'count': 371, 'mean': 854166.823787062, 'zones': 133305000300.0},
 {'count': 168, 'mean': 854348.412202381, 'zones': 133306010

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

In [ ]:
Previous runs:  
0:24:15.930678    